In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, BatchNormalization, Conv1D, Multiply, Activation, MaxPooling1D
from keras.regularizers import l2
from keras.optimizers import Adam, SGD

## Below path hardcoded. TODO: Change this
prefix_path = '..'

labels = pd.read_csv(prefix_path + '/train_kaggle.csv')

print('Labels', labels.describe())


X = []
y = []

max_len = 350
batch_size = 512





/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

Labels                  Id         label
count  30336.000000  30336.000000
mean   15167.500000      0.077664
std     8757.393219      0.267646
min        0.000000      0.000000
25%     7583.750000      0.000000
50%    15167.500000      0.000000
75%    22751.250000      0.000000
max    30335.000000      1.000000


/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.d

In [2]:
## For each sample in the file
for index, train_label in labels.iterrows():
    label = train_label['label']

    ## zeros_array used to keep the maximum number of sequences constant to max_len
    zeros_array = np.zeros((max_len, 40))

    ## features is a (N, 40) matrix
    features = np.load(prefix_path + '/train/train/' + str(train_label['Id']) + '.npy')
    
    ## We add it to zeros_array to make all samples as (400, 40) matrix
    zeros_array[0:len(features)] = features


    ## For each feature, we find average of all values and replace all NaN with that value
    for feature in range(40):
        average_value = np.average(zeros_array[:feature][np.nan_to_num(zeros_array[:feature]) != 0])
        zeros_array[:feature] = np.nan_to_num(zeros_array[:feature], average_value)

    X.append(zeros_array)
    y.append(label)

X = np.array(X)
y = np.array(y)

print('X Shape', X.shape)
print('y shape', y.shape)
print(X)


/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/numpy/lib/function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/mihir/pythonprojects/cs5228_project/cs5228/venv/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


X Shape (30336, 350, 40)
y shape (30336,)
[[[ 0.    0.    1.   ...  0.    0.    0.  ]
  [ 0.    0.    2.   ... 19.    0.    0.  ]
  [ 0.    0.    3.   ... 22.    0.    0.  ]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]]

 [[ 1.    0.    1.   ...  0.    0.    0.  ]
  [ 1.    0.    2.   ... 17.5   0.   36.44]
  [ 1.    0.    3.   ... 27.    0.    0.  ]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]]

 [[ 0.    0.    1.   ...  0.    0.    0.  ]
  [ 0.    0.    2.   ... 29.    0.    0.  ]
  [ 0.    0.    3.   ... 40.    0.   37.11]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]]

 ...

 [[ 0.    0.    1.   ...  0.    0.    0.  ]
  [ 0.    1.6   2.   ... 16.   31.4  37.2 ]
  [ 0.    0.    3.   ... 17.5   0.    0.  ]
  ..

In [3]:

sample_array = [[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]]
sample_array = np.array(sample_array)
print(sample_array.shape)
print('Total average', np.mean(sample_array))
print('Feats average', np.mean(sample_array, axis=(0, 1)))

(2, 2, 3)
Total average 3.5
Feats average [2.5 3.5 4.5]


In [4]:
num_samples, num_sequences, num_features = X.shape[0], X.shape[1], X.shape[2]
print("{} Samples, {} sequential samples, {} features".format(num_samples, num_sequences, num_features))
feature_wise_average = np.mean(X, axis=(0, 1))
assert feature_wise_average.shape[0] == num_features
print("Feature-wise average")
print(feature_wise_average)

for feature in range(num_features):
    feature_values = X[:, :, feature]
    #print('Feature {} ; NaN values: {} out of {}'.format(feature, np.count_nonzero(feature_values), feature_values.shape[0] * feature_values.shape[1]))
    print('Feature {} ; NaN values: {} out of {}'.format(feature, np.count_nonzero(~np.isnan(feature_values)), feature_values.shape[0] * feature_values.shape[1]))

30336 Samples, 350 sequential samples, 40 features
Feature-wise average
[        nan         nan  2.99179363         nan         nan         nan
         nan         nan         nan         nan         nan  6.8681197
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan  0.06232454         nan -5.93850188
         nan         nan         nan         nan]
Feature 0 ; NaN values: 10525041 out of 10617600
Feature 1 ; NaN values: 10432727 out of 10617600
Feature 2 ; NaN values: 10617600 out of 10617600
Feature 3 ; NaN values: 10587396 out of 10617600
Feature 4 ; NaN values: 10423339 out of 10617600
Feature 5 ; NaN values: 10425357 out of 10617600
Feature 6 ; NaN values: 10432825 out of 10617600
Feature 7 ; NaN values: 10525041 out of 10617600
Feature 8 ; NaN values: 10436829 out of 10617600


In [171]:
# X_1 = X[:, :, 36]
print(X)
for feature in range(num_features):
    X_1 = X[:, :, feature]
    print('feature : ', feature, X_1.shape, np.isnan(X_1).all())
    print(X_1)


[[[ 0.    0.    1.   ...  0.    0.    0.  ]
  [ 0.    0.    2.   ... 19.    0.    0.  ]
  [ 0.    0.    3.   ... 22.    0.    0.  ]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]]

 [[ 1.    0.    1.   ...  0.    0.    0.  ]
  [ 1.    0.    2.   ... 17.5   0.   36.44]
  [ 1.    0.    3.   ... 27.    0.    0.  ]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]]

 [[ 0.    0.    1.   ...  0.    0.    0.  ]
  [ 0.    0.    2.   ... 29.    0.    0.  ]
  [ 0.    0.    3.   ... 40.    0.   37.11]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]
  [ 0.    0.    0.   ...  0.    0.    0.  ]]

 ...

 [[ 0.    0.    1.   ...  0.    0.    0.  ]
  [ 0.    1.6   2.   ... 16.   31.4  37.2 ]
  [ 0.    0.    3.   ... 17.5   0.    0.  ]
  ...
  [ 0.    0.    0.   ...  0.    0.    0.

feature :  28 (30336, 350) False
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0. 81.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
feature :  29 (30336, 350) False
[[ 0. 95. 99. ...  0.  0.  0.]
 [ 0. 99. 98. ...  0.  0.  0.]
 [ 0. 96. 97. ...  0.  0.  0.]
 ...
 [ 0. 97. 94. ...  0.  0.  0.]
 [ 0. 93.  0. ...  0.  0.  0.]
 [ 0.  0. 97. ...  0.  0.  0.]]
feature :  30 (30336, 350) False
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0. 59.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
feature :  31 (30336, 350) False
[[ 0.   0.   0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]
 ...
 [ 0.  37.2  0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]
 [ 0.   0.   0.  ...  0.   0.   0. ]]
feature :  32 (30336, 350) False
[[ 0.   0.   0.  ...  0.   0.  

In [193]:
import sys
np.set_printoptions(threshold=10000)

print(y[10:20])

print(y[14])
print(X[14])
print(np.max(X[16][:, 2]))

print(y[7])
print(X[7])
print(X[7].shape)
print(np.max(X[7][:, 2]))


[1 0 0 0 1 0 0 1 0 0]
1
[[ 0.    0.    1.   ...  0.    0.    0.  ]
 [ 0.    2.4   2.   ... 11.   72.1  36.1 ]
 [ 0.    0.    3.   ...  9.5   0.   36.55]
 ...
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]]
37.0
0
[[ 0.    0.    1.   ...  0.    0.    0.  ]
 [ 0.    1.8   2.   ... 16.5   0.   36.06]
 [ 0.    1.8   3.   ... 16.    0.    0.  ]
 ...
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    0.  ]]
(350, 40)
40.0


In [25]:
A = X[:, :, 21]
print(A[1])
print(A[12])
print(A[123])
print(A[1234])
print(A[12345])
print(A[21])
print(A[1323])
print(A[7777])

[  0.   0.   0.   0.   0.   0.   0.   0.   0. 100.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 

In [7]:
X_2d = np.sum(X, axis=1)
print(X_2d.shape)
print(X_2d)

(30336, 40)
[[    nan     nan 1485.   ... 1241.       nan     nan]
 [  23.      2.5   276.   ...  270.5     0.    216.99]
 [   0.       nan 1176.   ...     nan     nan     nan]
 ...
 [   0.      7.4   630.   ...  701.5    31.4   261.7 ]
 [    nan     nan 1081.   ...  675.       nan     nan]
 [   0.      0.     36.   ...  125.      0.     71.9 ]]
